In [6]:
import sys

!{sys.executable} -m pip install numpy --user
!{sys.executable} -m pip install scipy --user
!{sys.executable} -m pip install ta --user
!{sys.executable} -m pip install datetime --user
!{sys.executable} -m pip install yfinance --user
!{sys.executable} -m pip install MetaTrader5 --user

In [ ]:
import ta
import numpy as np
import warnings
from datetime import datetime
import pandas as pd
import MetaTrader5 as mt5
warnings.filterwarnings("ignore")
mt5.initialize()
from ClassMT5 import *
import time


creds = {
    "path": "C:/Program Files/MetaTrader 5/terminal64.exe",
    "login": 3000012345,
    "pass": "tu_contraseña",
    "server": "Darwinex-Demo",
    "timeout": 60000,
    "portable": False
}


def SMA_strategy(symbol):
    
    # Import / Features engineering
    df = MT5.get_data(symbol, 70)
    
    # Creamos la Resistencia haciendo un rolling máximo
    df["SMA fast"] = df["close"].rolling(30).mean()

    # Creamos el Soporte haciendo un rolling mínimo
    df["SMA slow"] = df["close"].rolling(60).mean()


    # Señales
    condition_1_buy = df["SMA fast"].iloc[-1] > df["SMA slow"].iloc[-1]
    
    condition_1_sell = df["SMA fast"].iloc[-1] < df["SMA slow"].iloc[-1]
    
    buy = condition_1_buy
    sell = condition_1_sell
    

    return buy, sell


mt5.initialize()
# True = Live Trading and False = Screener
live = True

if live:
    current_account_info = mt5.account_info()
    print("------------------------------------------------------------------")
    print("Date: ", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    print(f"Balance: {current_account_info.balance} USD, \t"
          f"Equity: {current_account_info.equity} USD, \t"
          f"Profit: {current_account_info.profit} USD")
    print("------------------------------------------------------------------")

info_order = {
    "Ethereum": ["EURUSD", 0.1], #microlotes

}


start = datetime.now().strftime("%H:%M:%S") #"23:59:59"
while True:

    # Verificación para lanzar el algoritmo
    if datetime.now().weekday() not in (5,6): # Para hacer trading solamente de Lunes a Viernes
        is_time = datetime.now().strftime("%H:%M:%S") == start 
    else:
        is_time = False

    
    # Lanzamos el algoritmo
    if is_time:

        # Abrimos el trading
        for asset in info_order.keys():

            # Inicializamos la entrada de datos
            symbol = info_order[asset][0]
            lot = info_order[asset][1]

            # Creamos la señal
            buy, sell = SMA_strategy(symbol)

             # Ejecutamos el algoritmo
            if live:
                MT5.run(symbol, buy, sell,lot)

            else:
                print(f"Symbol: {symbol}\t"
                     f"Buy: {buy}\t"
                     f"Sell: {sell}")

------------------------------------------------------------------
Date:  2024-09-10 13:51:00
Balance: 100000.0 USD, 	Equity: 100000.0 USD, 	Profit: 0.0 USD
------------------------------------------------------------------
------------------------------------------------------------------
Date:  2024-09-10 13:51:00
SYMBOL: EURUSD
BUY: True 	  SHORT: False
POSITION: None 	 ID: None
OPEN LONG TRADE: Request executed
------------------------------------------------------------------
------------------------------------------------------------------
Date:  2024-09-10 13:51:00
SYMBOL: EURUSD
BUY: True 	  SHORT: False
POSITION: 0 	 ID: 2826732945
------------------------------------------------------------------
------------------------------------------------------------------
Date:  2024-09-10 13:51:00
SYMBOL: EURUSD
BUY: True 	  SHORT: False
POSITION: 0 	 ID: 2826732945
------------------------------------------------------------------
----------------------------------------------------

In [ ]:
MT5.account_info()